In [3]:
! pip install tensorflow keras

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import io
import requests

In [ ]:
# Load the data



data1 = pd.read_csv('input.csv',na_values='.')
                    
data2= pd.read_csv('output.csv',na_values='.')

# Extract the input features and target variables
X = data1.values
y = data2.values

print (data1.size)
print (data2.size)



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.01, random_state=42)

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)